In [1]:
from logging import root
from tweepy import API
from tweepy import Cursor
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from textblob import TextBlob
from wordcloud import WordCloud, STOPWORDS
import string
import numpy as np
import pandas as pd
import sys
sys.stdin.reconfigure(encoding='utf-8')
#sys.stdout.reconfigure(encoding='utf-8')

# Enter Username

In [2]:
screen_name="NHSEngland" #USERNAME

#TWITTER CREDENTIALS


In [3]:
# # # TWITTER CREDENTIALS # # #
CONSUMER_KEY="DSaamoVhWcE972jRrm5l4GaYE"
CONSUMER_SECRET="m7yLJ00fWzcmBAFigeM73Y4y5XEnYpaJmgCXoFajleeAFZS07g"
ACCESS_TOKEN="958716870890319872-WajrAitlXmIbxP0QVfmVieIaJk6ZktS"
ACCESS_TOKEN_SECRET="lBtni8TfYcYqFIL9Mhp9RtmmM3QorUq2t9IDJQGAajTS2"

#TWITTER CLIENT

In [4]:
# # # # TWITTER CLIENT # # # #
class TwitterClient():
    def __init__(self, twitter_user=None):
        self.auth = TwitterAuthenticator().authenticate_twitter_app()
        self.twitter_client = API(self.auth,wait_on_rate_limit=True,wait_on_rate_limit_notify=True)

        self.twitter_user = twitter_user

    def get_twitter_client_api(self):
        return self.twitter_client

    def get_user_timeline_tweets(self, num_tweets):
        tweets = []
        for tweet in Cursor(self.twitter_client.user_timeline, id=self.twitter_user, tweet_mode='extended',trunc=False).items(num_tweets):
            tweets.append(tweet)
        return tweets

    def get_friend_list(self, num_friends):
        friend_list = []
        for friend in Cursor(self.twitter_client.friends, id=self.twitter_user).items(num_friends):
            friend_list.append(friend)
        return friend_list

    def get_home_timeline_tweets(self, num_tweets):
        home_timeline_tweets = []
        for tweet in Cursor(self.twitter_client.home_timeline, id=self.twitter_user).items(num_tweets):
            home_timeline_tweets.append(tweet)
        return home_timeline_tweets

# TWITTER AUTHENTICATER

In [5]:
# # # # TWITTER AUTHENTICATER # # # #
class TwitterAuthenticator():

    def authenticate_twitter_app(self):
        auth = OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
        auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
        return auth


# TWITTER STREAMER

In [6]:
# # # # TWITTER STREAMER # # # #
class TwitterStreamer():
    """
    Class for streaming and processing live tweets.
    """

    def __init__(self):
        self.twitter_autenticator = TwitterAuthenticator()

    def stream_tweets(self, fetched_tweets_filename, hash_tag_list):
        # This handles Twitter authetification and the connection to Twitter Streaming API
        listener = TwitterListener(fetched_tweets_filename)
        auth = self.twitter_autenticator.authenticate_twitter_app()
        stream = Stream(auth, listener)

        # This line filter Twitter Streams to capture data by the keywords:
        stream.filter(track=hash_tag_list)



#TWITTER STREAM LISTENER

In [7]:
# # # # TWITTER STREAM LISTENER # # # #
class TwitterListener(StreamListener):
    """
    This is a basic listener that just prints received tweets to stdout.
    """

    def __init__(self, fetched_tweets_filename):
        self.fetched_tweets_filename = fetched_tweets_filename

    def on_data(self, data):
        try:
            print(data)
            with open(self.fetched_tweets_filename, 'a') as tf:
                tf.write(data)
            return True
        except BaseException as e:
            print("Error on_data %s" % str(e))
        return True

    def on_error(self, status):
        if status == 420:
            # Returning False on_data method in case rate limit occurs.
            return False
        print(status)



#TWEET ANALYZER

In [8]:
# # # # TWEET ANALYZER # # # #

class TweetAnalyzer():
    """
    Functionality for analyzing and categorizing content from tweets.
    """

    def tweets_to_data_frame(self, tweets):
        df = pd.DataFrame(data=[tweet.full_text for tweet in tweets], columns=['Tweets'])
        #df['id'] = np.array([tweet.id for tweet in tweets])
        #df['len'] = np.array([len(tweet.text) for tweet in tweets])
        #df['date'] = np.array([tweet.created_at for tweet in tweets])
        #df['source'] = np.array([tweet.source for tweet in tweets])
        #df['likes'] = np.array([tweet.favorite_count for tweet in tweets])
        #df['retweets'] = np.array([tweet.retweet_count for tweet in tweets])
        #df['friends'] = np.array([tweet.friend_count for tweet in tweets])

        return df

# -- Grab Data Extended --- (Input: Twitter's Username as Screen_Name)

In [9]:
twitter_client = TwitterClient()
tweet_analyzer = TweetAnalyzer()

api = twitter_client.get_twitter_client_api()
#screen_name="POTUS" #USERNAME
tweets = api.user_timeline(screen_name, count=200,tweet_mode='extended')

# print(dir(tweets[0]))
# print(tweets[0].retweet_count)
df = tweet_analyzer.tweets_to_data_frame(tweets)
df.to_csv("./{screenname}.csv".format(screenname=screen_name))

# hash_tag_list = ["SHIBA"]
# fetched_tweets_filename = "hash_tag_list1.csv"
# twitter_streamer = TwitterStreamer()

# tweets=twitter_streamer.stream_tweets(fetched_tweets_filename, hash_tag_list)
        
# df = tweet_analyzer.tweets_to_data_frame(tweets)
# df.to_csv("dogecoins.csv")

# tweet_analyzer.tweets_to_data_frame(twitter_streamer.stream_tweets(fetched_tweets_filename, hash_tag_list)).to_csv("dogecoins.csv")

#CLEANING DATA

In [10]:
import pandas as pd
import nltk
import re
import spacy
import nltk
#import preprocess_kgptalkie as ps
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
#%pip install scipy
#%pip install preprocess_kgptalkie
#%pip install preprocess_kgptalkie

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

# Regular Expression for URLS

In [11]:
raw_docs=pd.read_csv("./{screenname}.csv".format(screenname=screen_name))
# print(raw_docs['Tweets'].values[1]) #for reading line by line
# print(len(raw_docs))
import re
i=0
while i<len(raw_docs['Tweets']):
  raw_docs['Tweets'].values[i] = re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', " ", raw_docs['Tweets'].values[i])
  raw_docs['Tweets'].values[i]=re.sub("…\s", " ", raw_docs['Tweets'].values[i])
  raw_docs['Tweets'].values[i] = re.sub('\w*\d\w*', '', raw_docs['Tweets'].values[i])
  #raw_docs['Tweets'].values[i] = re.sub('’', '', raw_docs['Tweets'].values[i])
  #raw_docs['Tweets'].values[i] = re.sub('–', '', raw_docs['Tweets'].values[i])
  raw_docs['Tweets'].values[i] = re.sub('@[A-Za-z0-9_]+', '', raw_docs['Tweets'].values[i])
  raw_docs['Tweets'].values[i] = re.sub('#', '', raw_docs['Tweets'].values[i])
  raw_docs['Tweets'].values[i] = re.sub('RT', '', raw_docs['Tweets'].values[i])
  #raw_docs['Tweets'].values[i] = re.sub('—', '', raw_docs['Tweets'].values[i])
  #raw_docs['Tweets'].values[i] = re.sub('\n\n', '', raw_docs['Tweets'].values[i])
  #raw_docs['Tweets'].values[i] = re.sub('\n \n', '', raw_docs['Tweets'].values[i])
  #raw_docs['Tweets'].values[i] = re.sub('https?:\/\/[A-Za-z0-9\.\./]+', '', raw_docs['Tweets'].values[i])


  
  i=i+1
print(raw_docs["Tweets"].values[7])

 : Today we've published our What Good Looks Like guidance for digital transformation. On our blog, our national CIO @ set…


# Word Tokenize

In [12]:
# word tokenize
from nltk.tokenize import word_tokenize


tokenized_docs = [word_tokenize(doc) for doc in raw_docs['Tweets']]
print(tokenized_docs)


[['Looking', 'after', 'the', 'wellbeing', 'of', 'OurNHSPeople', 'means', 'they', 'can', 'better', 'look', 'after', 'patients', '.', 'If', 'you', "'re", 'finding', 'things', 'difficult', 'or', 'more', 'challenging', 'at', 'the', 'moment', ',', 'and', 'would', 'like', 'to', 'speak', 'to', 'someone', '—', 'please', 'call', ',', 'text', 'or', 'visit', '.', '📞', '💬'], ["'I", "'ve", 'had', 'my', 'COVID-', 'vaccine', '.', 'It', 'was', 'quick', 'and', 'easy', '.', 'I', "'m", 'really', 'pleased', 'to', 'have', 'had', 'it', 'done', '.', "'", '-year-old', 'Lemuella', 'has', 'had', 'her', 'NHSCOVIDVaccine', '.', 'Find', 'out', 'more', 'about', 'the', 'vaccine', 'and', 'how', 'you', 'can', 'get', 'yours', '.', '➡️'], [':', 'Delighted', 'that', 'the', 'NHS', 'will', 'now', 'be', 'able', 'to', 'offer', 'patients', 'inclisiran', ',', 'a', 'treatment', 'to', 'lower', 'cholesterol', ',', 'due', 'to', 'our', 'NHS-fir…'], [':', 'You', 'have', 'a', 'medicine', 'in', 'your', 'blood', 'but', 'you', 'probably

# Sentence Tokenization

In [13]:
#Sentence tokenization

from nltk.tokenize import sent_tokenize
sent_token = [sent_tokenize(doc) for doc in raw_docs['Tweets']]
print(sent_token)


[['Looking after the wellbeing of OurNHSPeople means they can better look after patients.', "If you're finding things difficult or more challenging at the moment, and would like to speak to someone — please call, text or visit  .", '📞 💬'], ["'I've had my COVID- vaccine.", 'It was quick and easy.', "I'm really pleased to have had it done.'", '-year-old Lemuella has had her NHSCOVIDVaccine.', 'Find out more about the vaccine and how you can get yours.', '➡️'], [' : Delighted that the NHS will now be able to offer patients inclisiran, a treatment to lower cholesterol, due to our NHS-fir…'], [' : You have a medicine in your blood but you probably don’t know about it.', '\U0001fa78🤔\n\nYour plasma could be the only thing that save…'], ['A new drug to lower cholesterol will be made available to hundreds of thousands of patients, thanks to a world-leading deal.', "It's estimated that Inclisiran could prevent , heart attacks and strokes, saving , lives within the next decade."], ['If you’re - 

# Remove Punctuation

In [14]:
#Remove punctuations
import re
import string

regex = re.compile('[%s]' % re.escape(string.punctuation))

tokenized_docs_no_punctuation = []

for review in tokenized_docs:
    new_review = []
    for token in review:
        new_token = regex.sub(u'', token)
        if not new_token == u'':
            new_review.append(new_token)
    
    tokenized_docs_no_punctuation.append(new_review)
    
print(tokenized_docs_no_punctuation)

[['Looking', 'after', 'the', 'wellbeing', 'of', 'OurNHSPeople', 'means', 'they', 'can', 'better', 'look', 'after', 'patients', 'If', 'you', 're', 'finding', 'things', 'difficult', 'or', 'more', 'challenging', 'at', 'the', 'moment', 'and', 'would', 'like', 'to', 'speak', 'to', 'someone', '—', 'please', 'call', 'text', 'or', 'visit', '📞', '💬'], ['I', 've', 'had', 'my', 'COVID', 'vaccine', 'It', 'was', 'quick', 'and', 'easy', 'I', 'm', 'really', 'pleased', 'to', 'have', 'had', 'it', 'done', 'yearold', 'Lemuella', 'has', 'had', 'her', 'NHSCOVIDVaccine', 'Find', 'out', 'more', 'about', 'the', 'vaccine', 'and', 'how', 'you', 'can', 'get', 'yours', '➡️'], ['Delighted', 'that', 'the', 'NHS', 'will', 'now', 'be', 'able', 'to', 'offer', 'patients', 'inclisiran', 'a', 'treatment', 'to', 'lower', 'cholesterol', 'due', 'to', 'our', 'NHSfir…'], ['You', 'have', 'a', 'medicine', 'in', 'your', 'blood', 'but', 'you', 'probably', 'don', '’', 't', 'know', 'about', 'it', '\U0001fa78🤔', 'Your', 'plasma', 'c

# Remove StopWords

In [15]:
from nltk.corpus import stopwords

tokenized_docs_no_stopwords = []

for doc in tokenized_docs_no_punctuation:
    new_term_vector = []
    for word in doc:
        if not word.lower() in stopwords.words('english'):
            new_term_vector.append(word)
    
    tokenized_docs_no_stopwords.append(new_term_vector)

print(tokenized_docs_no_stopwords)

[['Looking', 'wellbeing', 'OurNHSPeople', 'means', 'better', 'look', 'patients', 'finding', 'things', 'difficult', 'challenging', 'moment', 'would', 'like', 'speak', 'someone', '—', 'please', 'call', 'text', 'visit', '📞', '💬'], ['COVID', 'vaccine', 'quick', 'easy', 'really', 'pleased', 'done', 'yearold', 'Lemuella', 'NHSCOVIDVaccine', 'Find', 'vaccine', 'get', '➡️'], ['Delighted', 'NHS', 'able', 'offer', 'patients', 'inclisiran', 'treatment', 'lower', 'cholesterol', 'due', 'NHSfir…'], ['medicine', 'blood', 'probably', '’', 'know', '\U0001fa78🤔', 'plasma', 'could', 'thing', 'save…'], ['new', 'drug', 'lower', 'cholesterol', 'made', 'available', 'hundreds', 'thousands', 'patients', 'thanks', 'worldleading', 'deal', 'estimated', 'Inclisiran', 'could', 'prevent', 'heart', 'attacks', 'strokes', 'saving', 'lives', 'within', 'next', 'decade'], ['’', 'years', 'old', '’', 'eligible', 'NHSCOVIDVaccine', '’', 'yet', 'visit', 'find', 'nearest', 'walkin', 'centre'], ['Thank', 'colleagues', 'helped',

# Lemmatization

In [16]:
# Stemming and Lemmatization
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

porter = PorterStemmer()
wordnet = WordNetLemmatizer()

preprocessed_docs = []
preprocessed_docs1 = []

for doc in tokenized_docs_no_stopwords:
    final_doc = []
    
    for word in doc:
      # final_doc.append(porter.stem(word))
       #if word.lower() not in {"’","nt","‘","…","rt","—","–","nh"}:
       if word.lower() not in {"nt","…","rt","—","–","nh"}:
         final_doc.append(wordnet.lemmatize(word.lower()))
         
    preprocessed_docs.append(final_doc)
# for x in preprocessed_docs:
#   final_doc1=[]

#   for words in x:

#      final_doc1.append(porter.stem(words))

#   preprocessed_docs1.append(final_doc1)

print(preprocessed_docs)

[['looking', 'wellbeing', 'ournhspeople', 'mean', 'better', 'look', 'patient', 'finding', 'thing', 'difficult', 'challenging', 'moment', 'would', 'like', 'speak', 'someone', 'please', 'call', 'text', 'visit', '📞', '💬'], ['covid', 'vaccine', 'quick', 'easy', 'really', 'pleased', 'done', 'yearold', 'lemuella', 'nhscovidvaccine', 'find', 'vaccine', 'get', '➡️'], ['delighted', 'nh', 'able', 'offer', 'patient', 'inclisiran', 'treatment', 'lower', 'cholesterol', 'due', 'nhsfir…'], ['medicine', 'blood', 'probably', '’', 'know', '\U0001fa78🤔', 'plasma', 'could', 'thing', 'save…'], ['new', 'drug', 'lower', 'cholesterol', 'made', 'available', 'hundred', 'thousand', 'patient', 'thanks', 'worldleading', 'deal', 'estimated', 'inclisiran', 'could', 'prevent', 'heart', 'attack', 'stroke', 'saving', 'life', 'within', 'next', 'decade'], ['’', 'year', 'old', '’', 'eligible', 'nhscovidvaccine', '’', 'yet', 'visit', 'find', 'nearest', 'walkin', 'centre'], ['thank', 'colleague', 'helped', 'shape', 'guidanc

# Another Cleanning

In [17]:

with open("./{screenname}-Tweets.csv".format(screenname=screen_name),'w') as f:
    f.write('Tweets')
    f.writelines('\n')
    for sublist in preprocessed_docs:
        for item in sublist:
          if item.lower() not in {"’","nt","‘","…","rt","—","–","nh","youve","youre","weve","u","ve","re","“"}:
            f.write(item.lower() + ' ')
        f.writelines('\n')


# for sublist in preprocessed_docs:
#         for item in sublist:
#           if item.lower() in {"’","nt","‘","…","rt","—","–","nh","youve","weve","u","ve","re","“"}:
#             sublist.remove(item)

# preprocessed_docs

# Vectorization

In [ ]:
import spacy
!python -m spacy download en_core_web_lg
import en_core_web_lg
nlp = en_core_web_lg.load()

In [ ]:
def get_vec(x):
      doc = nlp(x)
      vec = doc.vector
      return vec

df=pd.read_csv("./{screenname}-Tweets.csv".format(screenname=screen_name))

df['vec'] = df['Tweets'].apply(lambda x: get_vec(x))
df.to_csv("./{screenname}-Final-Vectorized.csv".format(screenname=screen_name))
print(df.head())

In [ ]:
#!pip install wordcloud-fa
# import matplotlib.pyplot as plt
# from wordcloud_fa import WordCloudFa


# dd = pd.read_csv("./{screenname}-Final-Vectorized.csv".format(screenname=screen_name))

# text=' '.join(dd["Tweets"].tolist())

# wordcloud = WordCloudFa(include_numbers=True,width=1920,height=1080)

# wc = wordcloud.generate(text)
# image = wc.to_image()
# image.show()
# image.save('wordcloud.png')
# plt.figure(figsize=(10,10))
# plt.imshow(wordcloud)
# plt.axis('off')
# plt.tight_layout(pad=0)
# plt.show()

# WordCloud

In [ ]:

import wordcloud
from wordcloud import WordCloud
import matplotlib.pyplot as plt

dd = pd.read_csv("./{screenname}-Final-Vectorized.csv".format(screenname=screen_name))

text=' '.join(dd["Tweets"].tolist())
result = ''.join([i for i in text if not i.isdigit()])

wordcloud = WordCloud(width=1920,height=1080, background_color="black").generate(text)
# plt.figure(figsize=(10,10))
#plt.imshow(wordcloud, interpolation="bilinear")
# plt.axis("off")
# plt.show()
plt.figure(figsize=(10,10))
plt.imshow(wordcloud)
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()

# Correlation (Document to Matrix DTM)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import pickle
import pandas as pd

dd = pd.read_csv("./{screenname}-Final-Vectorized.csv".format(screenname=screen_name))
cv = CountVectorizer(stop_words='english')
data_cv = cv.fit_transform(dd['Tweets'])
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm.index = dd['Tweets'].index
result = ' '.join([i for i in data_dtm if not i.isdigit()])
#print(result)
#print("Number of Repeats in Tweets: \n" , data_dtm)

data_dtm.to_pickle("dtm.pkl") #Document-To-Matrix
pickle.dump(data_cv, open("data_cv.pkl", "wb"))
pickle.dump(cv, open("cv.pkl", "wb")) #CountVectorize
correlation = data_dtm.corr(method='pearson', min_periods=1)

print("Correlation: \n" , correlation)



In [ ]:
import pandas as pd

# data = pd.read_pickle('dtm.pkl')
data = pd.read_pickle('dtm.pkl')
data = data.transpose()
data

In [ ]:
top_dict = {}

for c in data:
    top = data[c].sort_values(ascending=False).head()
    top_dict[c]= list(zip(top.index, top.values))

top_dict

In [ ]:
for word, top_words in top_dict.items():
    print(word)
    print(', '.join([word for word, count in top_words]))
    print('---')

# Polarity and Subjectivity

In [ ]:
from textblob import *
dq = pd.read_csv("./{screenname}-Final-Vectorized.csv".format(screenname=screen_name))
#pol = lambda x: TextBlob(x).sentiment.polarity
#sub = lambda x: TextBlob(x).sentiment.subjectivity
#correct =  lambda x: TextBlob(x).correct

# print(dq['Tweets'].values[0])

def polarity(x):
  blob = TextBlob(x)
  return blob.sentiment.polarity

def subjectivity(x):
  blob = TextBlob(x)
  return blob.sentiment.subjectivity

  #print(blob.correct)
dq['polarity'] = dq['Tweets'].apply(lambda x: polarity(x))
dq['subjectivity'] = dq['Tweets'].apply(lambda x: subjectivity(x))
  #data['subjectivity'].values[i]=blob.sentiment.subjectivity
 # print(blob.sentiment)
  #data['polarity'] = pol(b)
  #data['subjectivity'] = i.apply(sub)

dq.to_csv("./{screenname}-Final-Vec-Pol-Subj.csv".format(screenname=screen_name))

import pickle
#data_dtm.to_pickle("dtm.pkl")
dq['Tweets'].to_pickle("dc.pkl")#Data Clean to Pickle

# Scatter Plot

In [ ]:
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [10, 8]

for item in dq.index:
    x = dq.polarity.loc[item]
    y = dq.subjectivity.loc[item]
    plt.scatter(x, y, color='blue')
   # plt.text(.001, .001,'happy', fontsize=10)
    plt.xlim(-1.1, 1.1) 
    plt.ylim(-.1, 1.1) 
    
plt.title("{screenname}: Sentiment Analysis".format(screenname=screen_name), fontsize=20)
plt.xlabel('<-- Negative ---------- Positive -->', fontsize=15)
plt.ylabel('<-- Facts ---------- Opinions -->', fontsize=15)
plt.hlines(0.5, -1.1, 1.1, colors=None, linestyles='solid', label='')
plt.vlines(0, -.1, 1.1, colors=None, linestyles='solid', label='')
plt.show()


In [ ]:
import matplotlib.pyplot as plt
from textblob import *
plt.rcParams['figure.figsize'] = [10, 8]

#mlob = TextBlob(text)

# x = mlob.sentiment.polarity
# y = mlob.sentiment.subjectivity

x=0
for i in range(0,dq['polarity'].values.shape[0]):
  x = dq['polarity'][i]+x
print("Polarity mean:",x/100)


y=0

for i in range(0,dq['subjectivity'].values.shape[0]):
  y = dq['subjectivity'][i]+y

print("subjectivity mean:",y/100)

plt.scatter(x/100, y/100, color='blue')
# plt.text(.001, .001,'happy', fontsize=10)
plt.xlim(-1.1, 1.1) 
plt.ylim(-.1, 1.1) 
    
plt.title("{screenname}: Sentiment Analysis All of it".format(screenname=screen_name), fontsize=20)
plt.xlabel('<-- Negative ---------- Positive -->', fontsize=15)
plt.ylabel('<-- Facts ---------- Opinions -->', fontsize=15)
plt.hlines(0.5, -1.1, 1.1, colors=None, linestyles='solid', label='')
plt.vlines(0, -.1, 1.1, colors=None, linestyles='solid', label='')
plt.show()


In [ ]:
dq['Tweets']

In [ ]:


# Show the plot for every tweet
plt.rcParams['figure.figsize'] = [20, 8]
plt.plot(dq['polarity'],label='Polarity')
plt.plot(dq['subjectivity'],label='Subjectivity')
plt.hlines(0.0, -1.0, 200, colors='red', linestyles='solid', label='')
plt.title("{screenname}".format(screenname=screen_name))
plt.legend()
plt.show()

In [ ]:
# Create function to show Negative-Neutral-Positive and add to dataset
def getAnalysis(score):
  if score < 0:
    return "Negative"
  if score == 0:
    return "Neutral"

  else: return "Positive"


da = pd.read_csv("./{screenname}-Final-Vec-Pol-Subj.csv".format(screenname=screen_name))

da['Analysis'] = da['polarity'].apply(getAnalysis)
# Delete a column by column number
# Delete column number 4 (index number 3 in data.columns)

da=da.drop(columns=['Unnamed: 0','Unnamed: 0.1'])


da.to_csv("./{screenname}-Final-Vec-Pol-Subj-Analysis.csv".format(screenname=screen_name))

# Print all positive tweets

In [ ]:
#Print all positive value

j=1
sortedDA= da.sort_values(by=['polarity'])

for i in range(0,sortedDA.shape[0]):
  if (sortedDA['Analysis'][i]=='Positive'):
    print(str(j)+ ') ' + sortedDA['Tweets'][i])
    print()
    j=j+1


In [ ]:
#Percentage of Negative and Positive Tweets

ptweets = da[da['Analysis']=='Positive']
negtweets=da[da['Analysis']=='Negative']
neutweets = da [da['Analysis']=='Neutral']

print("Positive Tweets:", round((ptweets.shape[0]/da.shape[0]) *100,1),"%")

print("Negative Tweets:", round((negtweets.shape[0]/da.shape[0]) *100,1),"%")

print("Neutral Tweets:", round((neutweets.shape[0]/da.shape[0]) *100,1),"%")

pos=ptweets.shape[0]/da.shape[0] *100
neg=negtweets.shape[0]/da.shape[0] *100
neut=neutweets.shape[0]/da.shape[0] *100

#Bar Plot
plt.rcParams['figure.figsize'] = [8, 6]
plt.title("Analysis")
plt.xlabel("Sentiment")
plt.ylabel("Counts")
da['Analysis'].value_counts().plot(kind='bar')

plt.show()

#Pie Plot

explode = (0,0.1,0)
labels = 'Positive', 'Negative','Neutral'
sizes=[pos,neg,neut]
colours = ['green','red','gold']
plt.pie(sizes,explode=explode,colors=colours,autopct='%1.1f%%',startangle=120)
plt.legend(labels,loc=(-0.05,0.05),shadow=True)
plt.axis('equal')
plt.savefig('TwitterSNA.png')

In [ ]:

plt.rcParams['figure.figsize'] = [10, 8]

for index, row in da.iterrows():
  if row['Analysis'] == 'Positive':
    plt.scatter(row['polarity'],row['subjectivity'],color='green')
  elif row['Analysis'] == 'Negative':
    plt.scatter(row['polarity'],row['subjectivity'],color='red')
  elif row['Analysis'] == 'Neutral':
    plt.scatter(row['polarity'],row['subjectivity'],color='blue')

plt.title("{screenname} Twitter Analysis".format(screenname=screen_name), fontsize=20)
plt.xlabel('Polarity', fontsize=15)
plt.ylabel('Subjectivity', fontsize=15)
# plt.hlines(0.5, -1.1, 1.1, colors=None, linestyles='solid', label='')
# plt.vlines(0, -.1, 1.1, colors=None, linestyles='solid', label='')
labels=['Positive','Neutral','Negative']
plt.legend((plt.scatter(row['polarity'],row['subjectivity'],color='green'), plt.scatter(row['polarity'],row['subjectivity'],color='blue'), plt.scatter(row['polarity'],row['subjectivity'],color='red')), ('Positive', 'Neutral', 'Negative'))
plt.show()

# Topic modeling Latent Dirichlet Allocation (LDA)

In [ ]:


# read our document-term matrix
import pandas as pd
import pickle

data = pd.read_pickle('dtm.pkl')
data



In [ ]:
from gensim import matutils, models
import scipy.sparse
tdm = data.transpose()
tdm

In [ ]:
sparse_counts = scipy.sparse.csr_matrix(tdm)
corpus = matutils.Sparse2Corpus(sparse_counts)


In [ ]:
cv = pickle.load(open("cv.pkl", "rb"))
id2word = dict((v, k) for k, v in cv.vocabulary_.items())
print(id2word)

# the number of topics and the number of passes.

In [ ]:
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=2, passes=100)
lda.print_topics()

In [ ]:
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=3, passes=100)
lda.print_topics()

In [ ]:
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=4, passes=100)
lda.print_topics()

# Topic Modeling - step 2 (Nouns Only)

In [ ]:


# create a function to pull out nouns from a string of text
from nltk import word_tokenize, pos_tag
nltk.download('averaged_perceptron_tagger')

def nouns(text):
    '''Given a string of text, tokenize the text and pull out only the nouns.'''
    is_noun = lambda pos: pos[:2] == 'NN'
    tokenized = word_tokenize(text)
    all_nouns = [word for (word, pos) in pos_tag(tokenized) if is_noun(pos)] 
    return ' '.join(all_nouns)



In [ ]:


# Read in the cleaned data, before the CountVectorizer step

data_clean = pd.read_pickle('dc.pkl')
data_clean



In [ ]:
# Apply the nouns function to the transcripts to filter only on nouns

# ddd = pd.read_csv("./{screenname}-Final-Vec-Pol-Subj.csv".format(screenname=screen_name))

# bigchunk=' '.join(ddd["Tweets"].tolist())
# chunk = str(bigchunk)
# nouns(chunk)

data_nouns = pd.DataFrame(data_clean.apply(nouns))
data_nouns['Tweets']

In [ ]:
# Create a new document-term matrix using only nouns
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer

# Re-add the additional stop words since we are recreating the document-term matrix
add_stop_words = ['like', 'im', 'know', 'just', 'dont', 'thats', 'right',
                  'youre', 'got', 'gonna', 'time', 'think', 'yeah', 'said']
stop_words = text.ENGLISH_STOP_WORDS.union(add_stop_words)

# Recreate a document-term matrix with only nouns
# cvn = CountVectorizer(stop_words=stop_words)
# data_cvn = cvn.fit_transform(data_nouns.index)
# data_dtmn = pd.DataFrame(data_cvn.toarray(), columns=cvn.get_feature_names())
# data_dtmn.index = data_nouns.index

# data_dtmn



cvn = CountVectorizer(stop_words=stop_words)
data_cvn = cvn.fit_transform(data_nouns['Tweets'])
data_dtmn = pd.DataFrame(data_cvn.toarray(), columns=cvn.get_feature_names())
data_dtmn.index = data_nouns['Tweets'].index

print(data_dtmn)
data_dtmn.to_pickle("dtmn.pkl")

In [ ]:

from scipy import *
# # Create the gensim corpus
# corpusn = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmn.transpose()))

# # Create the vocabulary dictionary
# id2wordn = dict((v, k) for k, v in cvn.vocabulary_.items())

sparse_countsn = scipy.sparse.csr_matrix(data_dtmn.transpose())
corpusn = matutils.Sparse2Corpus(sparse_countsn)
#corpusn=matutils.Sparse2Corpus(scipy.sparse.csc_matrix(data_dtmn.transpose()))
# Create the vocabulary dictionary
id2wordn = dict((v, k) for k, v in cvn.vocabulary_.items())
print(id2wordn)

In [ ]:
# Let's start with 2 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=2, id2word=id2wordn, passes=100)
ldan.print_topics()

In [ ]:
# Let's start with 2 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=3, id2word=id2wordn, passes=100)
ldan.print_topics()

In [ ]:
# Let's start with 2 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=4, id2word=id2wordn, passes=100)
ldan.print_topics()

# Topic Modeling - step 3 (Nouns and Adjectives)

In [ ]:
# Let's create a function to pull out nouns and adjectives from a string of text
def nouns_adj(text):
    '''Given a string of text, tokenize the text and pull out only the nouns and adjectives.'''
    is_noun_adj = lambda pos: pos[:2] == 'NN' or pos[:2] == 'JJ'
    tokenized = word_tokenize(text)
    nouns_adj = [word for (word, pos) in pos_tag(tokenized) if is_noun_adj(pos)] 
    return ' '.join(nouns_adj)

In [ ]:
# Apply the nouns function to the tweets to filter only on nouns
data_nouns_adj = pd.DataFrame(data_clean.apply(nouns_adj))
data_nouns_adj

In [ ]:
# Create a new document-term matrix using only nouns and adjectives, also remove common words with max_df
cvna = CountVectorizer(stop_words=stop_words, max_df=.8)
data_cvna = cvna.fit_transform(data_nouns_adj['Tweets'])
data_dtmna = pd.DataFrame(data_cvna.toarray(), columns=cvna.get_feature_names())
data_dtmna.index = data_nouns_adj.index
data_dtmna

In [ ]:
# Create the gensim corpus
corpusna = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmna.transpose()))

# Create the vocabulary dictionary
id2wordna = dict((v, k) for k, v in cvna.vocabulary_.items())

In [ ]:


# Let's start with 2 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=2, id2word=id2wordna, passes=100)
ldana.print_topics()



In [ ]:


# Let's start with 3 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=3, id2word=id2wordna, passes=100)
ldana.print_topics()



In [ ]:


# Let's start with 4 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=4, id2word=id2wordna, passes=100)
ldana.print_topics()



# Identify Topics

In [ ]:
# Final LDA model - Nouns and Adjectives
ldana = models.LdaModel(corpus=corpusna, num_topics=1, id2word=id2wordna, passes=100)
#ldana.print_topics()
ldana.show_topics()


In [ ]:
import wordcloud
from wordcloud import WordCloud
import matplotlib.pyplot as plt

dd = pd.read_csv("./{screenname}-Final-Vec-Pol-Subj-Analysis.csv".format(screenname=screen_name))

text=' '.join(dd["Tweets"].tolist())
result = ''.join([i for i in text if not i.isdigit()])

wordcloud = WordCloud(width=1920,height=1080, background_color="black",max_words=50).generate(text)
# plt.figure(figsize=(10,10))
# plt.imshow(wordcloud, interpolation="bilinear")
# plt.axis("off")
# plt.show()
plt.figure(figsize=(10,10))
plt.imshow(wordcloud)
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()

In [ ]:
from gensim import models
import gensim

text=preprocessed_docs
model = gensim.models.Word2Vec(text,size=100,max_vocab_size=5000)
words = list(model.wv.vocab)

print(model)
print(model['covid'])

# skipgram Model


In [ ]:
import numpy as np
from tqdm import tqdm_notebook
import re

# Data Prep

In [ ]:
def tokenize(text):
    # obtains tokens with a least 1 alphabet
    pattern = re.compile(r'[A-Za-z]+[\w^\']*|[\w^\']*[A-Za-z]+[\w^\']*')
    return pattern.findall(text.lower())

def mapping(tokens):
    word_to_id = dict()
    id_to_word = dict()

    for i, token in enumerate(set(tokens)):
        word_to_id[token] = i
        id_to_word[i] = token

    return word_to_id, id_to_word

def generate_training_data(tokens, word_to_id, window_size):
    N = len(tokens)
    X, Y = [], []

    for i in range(N):
        nbr_inds = list(range(max(0, i - window_size), i)) + \
                   list(range(i + 1, min(N, i + window_size + 1)))
        for j in nbr_inds:
            X.append(word_to_id[tokens[i]])
            Y.append(word_to_id[tokens[j]])
            
    X = np.array(X)
    X = np.expand_dims(X, axis=0)
    Y = np.array(Y)
    Y = np.expand_dims(Y, axis=0)
            
    return X, Y

In [ ]:
doc = "After the deduction of the costs of investing, " \
      "beating the stock market is a loser's game."
tokens = tokenize(str(text))
word_to_id, id_to_word = mapping(tokens)
X, Y = generate_training_data(tokens, word_to_id, 3)
vocab_size = len(id_to_word)
m = Y.shape[1]
# turn Y into one hot encoding
Y_one_hot = np.zeros((vocab_size, m))
Y_one_hot[Y.flatten(), np.arange(m)] = 1

In [ ]:
Y

# Initialization

In [ ]:
def initialize_wrd_emb(vocab_size, emb_size):
    """
    vocab_size: int. vocabulary size of your corpus or training data
    emb_size: int. word embedding size. How many dimensions to represent each vocabulary
    """
    WRD_EMB = np.random.randn(vocab_size, emb_size) * 0.01
    
    assert(WRD_EMB.shape == (vocab_size, emb_size))
    return WRD_EMB

def initialize_dense(input_size, output_size):
    """
    input_size: int. size of the input to the dense layer
    output_szie: int. size of the output out of the dense layer
    """
    W = np.random.randn(output_size, input_size) * 0.01
    
    assert(W.shape == (output_size, input_size))
    return W

def initialize_parameters(vocab_size, emb_size):
    WRD_EMB = initialize_wrd_emb(vocab_size, emb_size)
    W = initialize_dense(emb_size, vocab_size)
    
    parameters = {}
    parameters['WRD_EMB'] = WRD_EMB
    parameters['W'] = W
    
    return parameters

# Forward Propagation

In [ ]:
def ind_to_word_vecs(inds, parameters):
    """
    inds: numpy array. shape: (1, m)
    parameters: dict. weights to be trained
    """
    m = inds.shape[1]
    WRD_EMB = parameters['WRD_EMB']
    word_vec = WRD_EMB[inds.flatten(), :].T
    
    assert(word_vec.shape == (WRD_EMB.shape[1], m))
    
    return word_vec

def linear_dense(word_vec, parameters):
    """
    word_vec: numpy array. shape: (emb_size, m)
    parameters: dict. weights to be trained
    """
    m = word_vec.shape[1]
    W = parameters['W']
    Z = np.dot(W, word_vec)
    
    assert(Z.shape == (W.shape[0], m))
    
    return W, Z

def softmax(Z):
    """
    Z: output out of the dense layer. shape: (vocab_size, m)
    """
    softmax_out = np.divide(np.exp(Z), np.sum(np.exp(Z), axis=0, keepdims=True) + 0.001)
    assert(softmax_out.shape == Z.shape)

    return softmax_out

def forward_propagation(inds, parameters):
    word_vec = ind_to_word_vecs(inds, parameters)
    W, Z = linear_dense(word_vec, parameters)
    softmax_out = softmax(Z)
    
    caches = {}
    caches['inds'] = inds
    caches['word_vec'] = word_vec
    caches['W'] = W
    caches['Z'] = Z
    return softmax_out, caches

# Cross Entropy


In [ ]:
def cross_entropy(softmax_out, Y):
    """
    softmax_out: output out of softmax. shape: (vocab_size, m)
    """
    m = softmax_out.shape[1]
    cost = -(1 / m) * np.sum(np.log(softmax_out[Y.flatten(), np.arange(Y.shape[1])] + 0.001))
    return cost

# Backward Propagation

In [ ]:
def softmax_backward(Y, softmax_out):
    """
    Y: labels of training data. shape: (vocab_size, m)
    softmax_out: output out of softmax. shape: (vocab_size, m)
    """
    m = Y.shape[1]
    softmax_out[Y.flatten(), np.arange(m)] -= 1.0
    dL_dZ = softmax_out
    
    assert(dL_dZ.shape == softmax_out.shape)
    return dL_dZ

def dense_backward(dL_dZ, caches):
    """
    dL_dZ: shape: (vocab_size, m)
    caches: dict. results from each steps of forward propagation
    """
    W = caches['W']
    word_vec = caches['word_vec']
    m = word_vec.shape[1]
    
    dL_dW = (1 / m) * np.dot(dL_dZ, word_vec.T)
    dL_dword_vec = np.dot(W.T, dL_dZ)

    assert(W.shape == dL_dW.shape)
    assert(word_vec.shape == dL_dword_vec.shape)
    
    return dL_dW, dL_dword_vec

def backward_propagation(Y, softmax_out, caches):
    dL_dZ = softmax_backward(Y, softmax_out)
    dL_dW, dL_dword_vec = dense_backward(dL_dZ, caches)
    
    gradients = dict()
    gradients['dL_dZ'] = dL_dZ
    gradients['dL_dW'] = dL_dW
    gradients['dL_dword_vec'] = dL_dword_vec
    
    return gradients

def update_parameters(parameters, caches, gradients, learning_rate):
    vocab_size, emb_size = parameters['WRD_EMB'].shape
    inds = caches['inds']
    dL_dword_vec = gradients['dL_dword_vec']
    m = inds.shape[-1]
    
    parameters['WRD_EMB'][inds.flatten(), :] -= dL_dword_vec.T * learning_rate

    parameters['W'] -= learning_rate * gradients['dL_dW']

# MODEL

In [ ]:
from datetime import datetime

import matplotlib.pyplot as plt


def skipgram_model_training(X, Y, vocab_size, emb_size, learning_rate, epochs, batch_size=256, parameters=None, print_cost=False, plot_cost=True):
    costs = []
    m = X.shape[1]
    
    if parameters is None:
        parameters = initialize_parameters(vocab_size, emb_size)
    
    begin_time = datetime.now()
    for epoch in range(epochs):
        epoch_cost = 0
        batch_inds = list(range(0, m, batch_size))
        np.random.shuffle(batch_inds)
        for i in batch_inds:
            X_batch = X[:, i:i+batch_size]
            Y_batch = Y[:, i:i+batch_size]

            softmax_out, caches = forward_propagation(X_batch, parameters)
            cost = cross_entropy(softmax_out, Y_batch)
            gradients = backward_propagation(Y_batch, softmax_out, caches)
            update_parameters(parameters, caches, gradients, learning_rate)
            epoch_cost += np.squeeze(cost)
            
        costs.append(epoch_cost)
        if print_cost and epoch % (epochs // 500) == 0:
            print("Cost after epoch {}: {}".format(epoch, epoch_cost))
        if epoch % (epochs // 100) == 0:
            learning_rate *= 0.98
    end_time = datetime.now()
    print('training time: {}'.format(end_time - begin_time))
            
    if plot_cost:
        plt.plot(np.arange(epochs), costs)
        plt.xlabel('# of epochs')
        plt.ylabel('cost')
    return parameters

# TEST

In [ ]:

paras = skipgram_model_training(X, Y, vocab_size, 50, 0.05, 1000, batch_size=256, parameters=None, print_cost=True,plot_cost=True)

# Evaluate

In [ ]:
X_test = np.arange(vocab_size)
X_test = np.expand_dims(X_test, axis=0)
softmax_test, _ = forward_propagation(X_test, paras)
top_sorted_inds = np.argsort(softmax_test, axis=0)[-4:,:]

In [ ]:
for input_ind in range(vocab_size):
    input_word = id_to_word[input_ind]
    output_words = [id_to_word[output_ind] for output_ind in top_sorted_inds[::-1, input_ind]]
    print("{}'s neighbor words: {}".format(input_word, output_words))

In [ ]:
import pandas as pd

data = pd.read_pickle('dc.pkl')
data

#  Experiment: Tweet generator from last tweets

# Word2Vec

In [ ]:

from gensim.models import Word2Vec
# define training data
# sentences = [['this', 'is', 'the', 'first', 'sentence', 'for', 'word2vec'],
# 			['this', 'is', 'the', 'second', 'sentence'],
# 			['yet', 'another', 'sentence'],
# 			['one', 'more', 'sentence'],
# 			['and', 'the', 'final', 'sentence']]
sentences = text

# train model
model = Word2Vec(sentences, min_count=1)
# summarize the loaded model
print(model)
# summarize vocabulary
words = list(model.wv.vocab)
print(words)
# access vector for one word
#print(model['america'])
# save model
model.save('model.bin')
# load model
new_model = Word2Vec.load('model.bin')
print(new_model)


In [ ]:
	X = model[model.wv.vocab]

In [ ]:
from sklearn.decomposition import PCA
from matplotlib import pyplot
# define training data
# sentences = [['this', 'is', 'the', 'first', 'sentence', 'for', 'word2vec'],
# 			['this', 'is', 'the', 'second', 'sentence'],
# 			['yet', 'another', 'sentence'],
# 			['one', 'more', 'sentence'],
# 			['and', 'the', 'final', 'sentence']]
sentences = preprocessed_docs
# train model
model = Word2Vec(sentences, min_count=1)
# fit a 2d PCA model to the vectors
X = model[model.wv.vocab]
pca = PCA(n_components=10)
print(X.shape)
result = pca.fit_transform(X)
print(result.shape)
# create a scatter plot of the projection
plt.rcParams['figure.figsize'] = [30, 10]
pyplot.scatter(result[:, 0], result[:, 1])
words = list(model.wv.vocab)
for i, word in enumerate(words):
	pyplot.annotate(word, xy=(result[i, 0], result[i, 1]))
pyplot.show()

In [ ]:
# result = model.most_similar(positive=['people'], negative=['today'], topn=1)
# similar = model.most_similar('today', topn=1)
# print(result,'\n')
# print(similar,'\n\n')

In [ ]:
import csv
#file1 = open("MyFile.txt","a")
with open("MyFile.txt", "w") as my_output_file:
    with open("./{screenname}-Tweets.csv".format(screenname=screen_name), "r") as my_input_file:
        [ my_output_file.write(" ".join(row)+'\n') for row in csv.reader(my_input_file)]
    my_output_file.close()

# Create Model for Predicting Next Tweet Sentiment based on previous tweets.

In [ ]:
def Score(score):
  if score=='Negative':
    return -1
  elif score=='Neutral':
    return 0
  elif score=='Positive':
    return 1
    #Get Sentiment

def getSentimentforpred(score):
  if score<0:
    return 'Negative'
  if score==0:
    return 'Neutral'
  else:
    return 'Positive'


In [ ]:
#Function for Clean Tweets 
def cleanTwtforpred(twt):
  twt=re.sub('#[A-Za-z0-9]+', '', twt)
  twt=re.sub('\\n','',twt)
  twt=re.sub('https?:\/\/\S+','',twt)
  twt=re.sub('#','',twt)
  return twt

In [ ]:
#Function for SUBJECTIVITY and POLARITY

def getSubjectivityforpred(twt):
  return TextBlob(twt).sentiment.subjectivity


def getPolarityforpred(twt):
  return TextBlob(twt).sentiment.polarity


In [ ]:

newdf = pd.read_csv("./{screenname}.csv".format(screenname=screen_name))
#Clean
newdf['Cleaned_Tweets'] = newdf['Tweets'].apply(cleanTwtforpred)
newdf['Subjectivity'] = newdf['Cleaned_Tweets'].apply(getSubjectivityforpred)
newdf['Polarity'] = newdf['Cleaned_Tweets'].apply(getPolarityforpred)
newdf['Sentiment'] = newdf['Polarity'].apply(getSentimentforpred)

newdf['Score'] = newdf['Sentiment'].apply(Score)

In [ ]:
print(newdf['Cleaned_Tweets'].values[2],newdf['Score'].values[2])


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC,LinearSVC
from sklearn.metrics import classification_report

In [ ]:
X=newdf['Cleaned_Tweets']

tfidf = TfidfVectorizer(max_features=10000, ngram_range=(1,2))
X=tfidf.fit_transform(X)
y=newdf['Score']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=0)

In [ ]:
X_train.shape , X_test.shape

In [ ]:
clf=LinearSVC()
clf.fit(X_train,y_train)

In [ ]:
y_pred=clf.predict(X_test)


In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
NewTweet='I look forward to hosting President Ghani and Chairman Abdullah at the White House this afternoon. As the U.S. military drawdown continues, we affirm our enduring support for the Afghan people.'

vec=tfidf.transform([NewTweet])


In [ ]:
print('Model Score: ',clf.predict(vec))


# Custom Sentiment Analysis


In [ ]:
import csv
#file1 = open("MyFile.txt","a")
with open("MyFile.txt", "w") as my_output_file:
    with open("./{screenname}-Tweets.csv".format(screenname=screen_name), "r") as my_input_file:
        [ my_output_file.write(" ".join(row)+'\n') for row in csv.reader(my_input_file)]
    my_output_file.close()

In [ ]:
import string
from collections import Counter

import matplotlib.pyplot as plt

# reading text file
text = open("MyFile.txt", encoding="utf-8").read()

# converting to lowercase
lower_case = text.lower()

# Removing punctuations
cleaned_text = lower_case.translate(str.maketrans('', '', string.punctuation))

# splitting text into words
tokenized_words = cleaned_text.split()

stop_words = ["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself",
              "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself",
              "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these",
              "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do",
              "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while",
              "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before",
              "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again",
              "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each",
              "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than",
              "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]

# Removing stop words from the tokenized words list
final_words = []
for word in tokenized_words:
    if word not in stop_words:
        final_words.append(word)



emotion_list = []
with open('emotions.txt', 'r') as file:
    for line in file:
        clear_line = line.replace("\n", '').replace(",", '').replace("'", '').strip()
        word, emotion = clear_line.split(':')

        if word in final_words:
            emotion_list.append(emotion)

print(emotion_list)
w = Counter(emotion_list)
print(w)

# Plotting the emotions on the graph

fig, ax1 = plt.subplots()
ax1.bar(w.keys(), w.values())
plt.xticks(size = 24)
plt.yticks(size = 24)
plt.xlabel('Emotions', size = 18)
plt.ylabel('Counts', size = 18)
fig.autofmt_xdate()
plt.savefig('Custom-Sentiment-Analysis.png')
plt.show()

In [ ]:
import string
from collections import Counter
nltk.download('vader_lexicon')
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

text = open('MyFile.txt', encoding='utf-8').read()
lower_case = text.lower()
cleaned_text = lower_case.translate(str.maketrans('', '', string.punctuation))

# Using word_tokenize because it's faster than split()
tokenized_words = word_tokenize(cleaned_text, "english")

# Removing Stop Words
final_words = []
for word in tokenized_words:
    if word not in stopwords.words('english'):
        final_words.append(word)

# Lemmatization - From plural to single + Base form of a word (example better-> good)
lemma_words = []
for word in final_words:
    word = WordNetLemmatizer().lemmatize(word)
    lemma_words.append(word)

emotion_list = []
with open('emotions.txt', 'r') as file:
    for line in file:
        clear_line = line.replace("\n", '').replace(",", '').replace("'", '').strip()
        word, emotion = clear_line.split(':')

        if word in lemma_words:
            emotion_list.append(emotion)

print(emotion_list)
w = Counter(emotion_list)
print(w)


def sentiment_analyse(sentiment_text):
    score = SentimentIntensityAnalyzer().polarity_scores(sentiment_text)
    if score['neg'] > score['pos']:
        print("Overal Score: Negative Sentiment")
    elif score['neg'] < score['pos']:
        print("Overal Score: Positive Sentiment")
    else:
        print("Overal Score: Neutral Sentiment")


sentiment_analyse(cleaned_text)

fig, ax1 = plt.subplots()
ax1.bar(w.keys(), w.values())
fig.autofmt_xdate()
plt.savefig('graph.png')
plt.show()

# Experiment: Tweet Generator from previous tweets

In [ ]:
from collections import defaultdict

def markov_chain(text):
    '''The input is a string of text and the output will be a dictionary with each word as
       a key and each value as the list of words that come after the key in the text.'''
    
    # Tokenize the text by word, though including punctuation
    words = text.split(' ')
    
    # Initialize a default dictionary to hold all of the words and next words
    m_dict = defaultdict(list)
    
    # Create a zipped list of all of the word pairs and put them in word: list of next words format
    for current_word, next_word in zip(words[0:-1], words[1:]):
        m_dict[current_word].append(next_word)

    # Convert the default dict back into a dictionary
    m_dict = dict(m_dict)
    return m_dict

    # Create the dictionary Tweets
#all= "The input is a string of text and the output will be a dictionary with each word as a key and each value as the list of words that come after the key in the text."
#all = pd.read_csv('POTUS-Tweets.csv')
# all=dd["Tweets"]
# chunk=' '.join(dd["Tweets"].tolist())
all=data
chunk=' '.join(data.tolist())
alldict = markov_chain(chunk)

print(alldict)

import random

def generate_sentence(chain, count=20):
    '''Input a dictionary in the format of key = current word, value = list of next words
       along with the number of words you would like to see in your generated sentence.'''

    # Capitalize the first word
    word1 = random.choice(list(chain.keys()))
    sentence = word1.capitalize()

    # Generate the second word from the value list. Set the new word as the first word. Repeat.
    for i in range(count-1):
        word2 = random.choice(chain[word1])
        word1 = word2
        sentence += ' ' + word2

    # End it with a period
    sentence += '.'
    return(sentence)

generate_sentence(alldict)

# Experiment: Working on HashTags, 1000 recent (live) tweets that are using certain hashtag

In [ ]:
import tweepy as tw
# Authenticate
auth = tw.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
# Set Tokens
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
# Instantiate API
api = tw.API(auth, wait_on_rate_limit=True)



In [ ]:


hashtag = "#crypto"
query = tw.Cursor(api.search, q=hashtag, lang='en',tweet_mode='extended').items(1000)
tweets = [{'Tweet':tweet.full_text, 'Timestamp':tweet.created_at} for tweet in query]
print(tweets)

# id = 1408457960024690690
  
# # fetching the status with extended tweet_mode
# status = api.get_status(id, tweet_mode = "extended",counts=10)
  
# # fetching the full_text attribute
# full_text = status.full_text 
  
# print("The text of the status is : \n\n" + full_text)





In [ ]:
df = pd.DataFrame.from_dict(tweets)
df.head()

In [ ]:
bitcoin_handle = ['BitCoin','bitcoin','btc' 'BTC', 'bit coin\'s']
dogecoin_handle = ['dogecoin','doge coin','Dogecoin','DG\'s']

In [ ]:
def identify_subject(tweet, refs):
    flag = 0 
    for ref in refs:
        if tweet.find(ref) != -1:
            flag = 1
    return flag

df['Bitcoin'] = df['Tweet'].apply(lambda x: identify_subject(x, bitcoin_handle)) 
df['Dogecoin'] = df['Tweet'].apply(lambda x: identify_subject(x, dogecoin_handle))
df.head(10)

In [ ]:
# Import stopwords
import nltk
from nltk.corpus import stopwords

# Import textblob
from textblob import Word, TextBlob

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')
stop_words = stopwords.words('english')
custom_stopwords = ['RT', '#crypto']

In [ ]:
def preprocess_tweets(tweet, custom_stopwords):
    processed_tweet = tweet
    processed_tweet.replace('[^\w\s]', '')
    processed_tweet = " ".join(word for word in processed_tweet.split() if word not in stop_words)
    processed_tweet = " ".join(word for word in processed_tweet.split() if word not in custom_stopwords)
    processed_tweet = " ".join(Word(word).lemmatize() for word in processed_tweet.split())
    return(processed_tweet)

df['Processed Tweet'] = df['Tweet'].apply(lambda x: preprocess_tweets(x, custom_stopwords))
df.head()

In [ ]:
print('Base review\n', df['Tweet'][0])
print('\n------------------------------------\n')
print('Cleaned and lemmatized review\n', df['Processed Tweet'][0])

In [ ]:


# Calculate polarity
df['polarity'] = df['Processed Tweet'].apply(lambda x: TextBlob(x).sentiment[0])
df['subjectivity'] = df['Processed Tweet'].apply(lambda x: TextBlob(x).sentiment[1])
df[['Processed Tweet', 'Dogecoin', 'Bitcoin', 'polarity', 'subjectivity']].head()



In [ ]:
display(df[df['Bitcoin']==1][['Bitcoin','polarity','subjectivity']].groupby('Bitcoin').agg([np.mean, np.max, np.min, np.median]))
df[df['Dogecoin']==1][['Dogecoin','polarity','subjectivity']].groupby('Dogecoin').agg([np.mean, np.max, np.min, np.median])

In [ ]:
doge = df[df['Dogecoin']==1][['Timestamp', 'polarity']]
doge = doge.sort_values(by='Timestamp', ascending=True)
doge['MA Polarity'] = doge.polarity.rolling(10, min_periods=3).mean()

bit = df[df['Bitcoin']==1][['Timestamp', 'polarity']]
bit = bit.sort_values(by='Timestamp', ascending=True)
bit['MA Polarity'] = bit.polarity.rolling(10, min_periods=3).mean()

In [ ]:
doge.head()

In [ ]:
import matplotlib.pyplot as plt


btcarmy = 'red'
dgarmy = 'blue'
fig, axes = plt.subplots(1, 1, figsize=(13, 10))

# axes[0].plot(doge['Timestamp'], doge['MA Polarity'])
# axes[0].set_title("\n".join(["Dogecoin Polarity"]))
# axes[1].plot(bit['Timestamp'], bit['MA Polarity'], color='red')
# axes[1].set_title("\n".join(["Bitcoin Polarity"]))

plt.plot(doge['MA Polarity'], color='blue', label='Dogecoin')
plt.plot(bit['MA Polarity'], color='red', label='Bitcoin')

fig.suptitle("\n".join(["Crypto Analysis"]), y=0.98)
plt.legend()
plt.show()



In [ ]:
import tweepy as tw
import pandas as pd
from textblob import *
import re
# Authenticate
auth = tw.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
# Set Tokens
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
# Instantiate API
api = tw.API(auth, wait_on_rate_limit=True)



In [ ]:
search_term='# -filter:retweets'
tweets = tw.Cursor(api.search, q=search_term,lang='en', tweet_mode='extended').items(1000)
###NOTE: you can give a date to Cursor for example: since='2021-08-27'
all_tweets= [tweet.full_text for tweet in tweets]


In [ ]:
#Create Data Frame to Store tweets

tdf=pd.DataFrame(all_tweets , columns=['Tweets'])
tdf.head(5)

In [ ]:
#Function for Clean Tweets 
def cleanTwt(twt):
  twt=re.sub('#bitcoin','bitcoin', twt)
  twt=re.sub('#Bitcoin','Bitcoin', twt)
  twt=re.sub('#[A-Za-z0-9]+', '', twt)
  twt=re.sub('\\n','',twt)
  twt=re.sub('https?:\/\/\S+','',twt)
  twt=re.sub('#','',twt)
  return twt


In [ ]:
#Clean
tdf['Cleaned_Tweets'] = tdf['Tweets'].apply(cleanTwt)

tdf.head()

In [ ]:
#Function for SUBJECTIVITY and POLARITY

def getSubjectivity(twt):
  return TextBlob(twt).sentiment.subjectivity


def getPolarity(twt):
  return TextBlob(twt).sentiment.polarity

tdf['Subjectivity'] = tdf['Cleaned_Tweets'].apply(getSubjectivity)
tdf['Polarity'] = tdf['Cleaned_Tweets'].apply(getPolarity)

In [ ]:
#Get Sentiment

def getSentiment(score):
  if score<0:
    return 'Negative'
  if score==0:
    return 'Neutral'
  else:
    return 'Positive'



In [ ]:
#Create column to save score sentiment

tdf['Sentiment'] = tdf['Polarity'].apply(getSentiment)



In [ ]:
#create Scatter plot
import matplotlib.pyplot as plt

plt.figure(figsize=(8,6))

for i in range(0, tdf.shape[0]):
  plt.scatter(tdf['Polarity'][i], tdf['Subjectivity'][i], color='purple')
plt.title('Bitcoin Sentiment Analysis')
plt.xlabel('Negative <--------> Positive')
plt.ylabel('Facts <--------> Opinion')
plt.hlines(0.5,-1.1,1.1,colors='red')
plt.vlines(0,-.1,1,colors='red')
plt.show()

In [ ]:
tdf['Sentiment'].value_counts().plot(kind='bar')
plt.title('Bitcoin Sentiment Analysis')
plt.xlabel('Polarity')
plt.ylabel('Counts')
plt.show()